In [1]:
import pandas as pd
import numpy as np

In [2]:
n_classes = 120
n_negatives = 100_000

n_positives = np.loadtxt('../../data/raw/num_positives.txt', dtype=int)
assert n_positives.size == n_classes

with open('../../data/raw/smiles.txt', 'r') as f:
    smiles = f.readlines()
    
smiles = [s.strip() for s in smiles]
assert len(smiles) == n_negatives + n_positives.sum()

neg_smiles = smiles[:n_negatives]
smiles = smiles[n_negatives:]
pos_smiles = []
for class_ind in range(n_classes):
    tmp_n_positives = n_positives[class_ind]
    
    pos_smiles.append(smiles[:tmp_n_positives])
    smiles = smiles[tmp_n_positives:]

In [3]:
group_size = 10
assert n_classes % group_size == 0

np.random.seed(group_size)
shuffled_classes = np.random.permutation(np.arange(n_classes))
np.savetxt(f'../../data/split/{group_size}/shuffled_classes.txt', shuffled_classes, fmt='%d')

print(shuffled_classes)

[ 45  79  56  85  99  41  81 110  48   2  53  52  94  43  80   1  10  35
  47  26  68  82  37  19  14  39  38  63 104 111   3  75  70 108  97 115
 116  67  61  83  84  91  76  42  21   6  95 112 103  71 107  34  59  50
  55  60  66   4   5  44  24  32  58  20  98 109   7  46  17  74  96 117
 119 114 102  23  22  92  18  27 105  57  31  65  12  87  30  86  90  25
  13  69  77 101  51  49  78  72  33  62  88  54  11 118  16  36  40 106
   0  73   8  29  93  89  28 113  64  15 100   9]


In [4]:
for problem_ind in range(n_classes // group_size):
    tmp_smiles = []
    labels = []
    
    tmp_smiles += neg_smiles
    labels += [0] * n_negatives
    
    for class_ind in range(group_size):
        tmp_class = shuffled_classes[class_ind]
        
        tmp_smiles += pos_smiles[tmp_class]
        labels += [class_ind + 1] * n_positives[tmp_class]
        
        # with open(f'../../data/split/{group_size}/problem{problem_ind}.txt', 'w') as f:
        #     f.writelines(tmp_smiles)
        
        df = pd.DataFrame({'smiles': tmp_smiles, 'class': labels})
        df.to_csv(f'../../data/split/{group_size}/problem{problem_ind}.csv', index=False)

    shuffled_classes = shuffled_classes[group_size:]